In [4]:
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI

In [6]:
loader = CSVLoader(file_path="../data/data.csv")
data = loader.load()

In [7]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(data)
db = FAISS.from_documents(documents, OpenAIEmbeddings())

In [8]:
query = "Who should I pick as promotor for my engineering thesis 'Grounding Modal and Non-Modal Statements in Artificial Cognitive Agents Using Fuzzy-Linguistic Concepts?'"
docs = db.similarity_search(query)
print(docs[0].page_content)

autor: Grzegorz Popek
zainteresowania: artificial cognitionmulti-agent systems
opis pracy: "To enable artificial systems to meaningfully use a semantic language of communication is one of the long-term and key targets not only in the field of artificial cognitive agents
None: but also of AI research in general. Given existing solutions for grounding of modal statements of a language of communication and an idea to model internal concepts of the agent as zadehian fuzzy-linguistic concepts,this paper shows how to meaningfully combine the two within a single framework. An accomplished goal is a model for grounding of modal and non-modal statements of a language of communication based on concepts modelled internally as fuzzy sets spanned over the domain of observation. This paper describes a way in which fuzzy-linguistic concepts are activated by perceptual inputs and how an agents grounds respective non-modal statements. Further,an agent supposed to describe an unobserved part of the"


In [9]:
query = "Who should I pick as promotor for my engineering thesis The Role of Recomendation Systems in Ecommerce?"
docs = db.similarity_search(query)
print(docs[0].page_content)

autor: Rafał Palak
zainteresowania: Recommendation Systems Collective Intelligence Machine Learning
opis pracy: Nowadays, collective intelligence becomes more and more popular. Despite its high usability, many aspects of collective intelligence stay unexplored. Many companies have recognized the potential of collective intelligence and have begun using it. Prediction markets are the real life implementation of collective intelligence. The fact that prediction markets outperform experts makes it a great tool for predicting the future. In this paper, we try to answer important questions that have to be asked before the creation of a prediction market e. g. “What factors influence the prediction market error and how could this be minimized?”. This paper treats the problems more broadly. Therefore, the areas of collective intelligence that have a strong influence on prediction markets are also included in the problem analysis.


In [10]:
retriever = db.as_retriever(search_kwargs={"k": 5})

In [11]:
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(temperature=0.2),
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
)

d:\conda_folder\envs\promochator\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [12]:
prompt_template = """
You are an expert in providing information about thesis supervisors at Politechnika Wrocławska.
When given a user question about the most suitable thesis supervisor for their project, provide the supervisor's name along with their research interests.

User Question:
{question}

Response:
"""

In [15]:
question = "Who should I pick as promotor for my engineering thesis 'Grounding Modal and Non-Modal Statements in Artificial Cognitive Agents Using Fuzzy-Linguistic Concepts?"
formatted_prompt = prompt_template.format(question=question)
print(qa_chain.invoke(formatted_prompt)["result"])


The most suitable thesis supervisor for this project would be Grzegorz Popek, whose research interests include artificial cognition and multi-agent systems.


In [16]:
question = "Who should I pick as promotor for my engineering thesis The Role of Recomendation Systems in Ecommerce?"
formatted_prompt = prompt_template.format(question=question)
print(qa_chain.invoke(formatted_prompt)["result"])


Based on your project topic, I would recommend considering Rafał Palak as your thesis supervisor. His research interests include Recommendation Systems, Collective Intelligence, and Machine Learning.
